In [2]:
from skimage.data import imread
from skimage.io import imshow,imsave
from skimage import img_as_float
import pandas as pd
import numpy as np
import cv2
from os.path import join
from skimage.util import crop
from skimage.transform import rotate
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pickle

In [3]:
label_dir = '/home/ryan/cs/kaggle/ncfm/point_annotations'
json_names = ['lag_labels.json',
    'alb_labels.json',
              'bet_labels.json',
              'dol_labels.json',
              
              'other_labels.json',
              'shark_labels.json',
              'yft_labels.json',]

label_files = []
for n in json_names:
    label_files.append(join(label_dir, n))

img_dir = '/home/ryan/cs/datasets/ncfm/train'



save_path = '/home/ryan/cs/datasets/ncfm/aligner_train.pkl'

img_out_size = 224

In [3]:
def get_angle(row):
    if len(row['annotations']) < 2:
        return float('NaN')
    p1 = row['annotations'][0]
    p2 = row['annotations'][1]
    return deg_angle_between(p1['x'],p1['y'], p2['x'], p2['y'])

def deg_angle_between(x1,y1,x2,y2):
    from math import atan2, degrees, pi
    dx = x2 - x1
    dy = y2 - y1
    rads = atan2(-dy,dx)
    rads %= 2*pi
    degs = degrees(rads)
    if degs > 180:
        degs -= 360
    return(-1*degs)

In [4]:
def get_img(row):
    if len(row['annotations']) < 2:
        return float('NaN')
    p1 = row['annotations'][0]
    p2 = row['annotations'][1]

    img = imread(join(img_dir,row['filename']))

    boxed = get_cropped_fish(img, 
        int(p1['x']), int(p1['y']), int(p2['x']), int(p2['y']))

    return boxed
    

def get_cropped_fish(img,x1,y1,x2,y2):
    (h,w) = img.shape[:2]
    #calculate center and angle
    center = ( (x1+x2) / 2,(y1+y2) / 2)
    angle = np.floor(-deg_angle_between(x1,y1,x2,y2))
    #print('angle=' +str(angle) + ' ')
    #print('center=' +str(center))
    
    #M = cv2.getRotationMatrix2D(center, angle, 1.0)
    #rotated = cv2.warpAffine(img, M, (w, h))
    
    fish_length = np.sqrt((x1-x2)**2+(y1-y2)**2)
    
    cropped = img[(max((center[1]-int(fish_length/1.8)),0)):(max((center[1]+int(fish_length/1.8)),0)) ,
                      (max((center[0]-int(fish_length/1.8)),0)):(max((center[0]+int(fish_length/1.8)),0))]
    resized = resize(cropped,(img_out_size,img_out_size))
    return resized

In [14]:
labels = None
for i in range(len(label_files)):
    f_labels = pd.read_json(label_files[i])
    f_labels['fish'] = i
    if labels is None:
        labels = f_labels
    else:
        labels = labels.append(f_labels)

labels['rotation'] = labels.apply(get_angle, axis=1)
labels = labels.dropna()
print 'Calculated rotations.'

Calculated rotations.


In [15]:
images = np.zeros((len(labels), img_out_size, img_out_size, 3))
counter = 0
for _, row in labels.iterrows():
    images[counter] = get_img(row)
    counter += 1

In [16]:
print len(images)
print len(labels['rotation'])

3299
3299


In [17]:
train_set = {'images': images, 'labels': labels['rotation'].values}
pickle.dump(train_set, open(save_path, 'wb'))